# Accelerate SetFit Models with 🤗 Optimum Intel

In this notebook, we'll show how you can use 🤗 [Optimum Intel](https://github.com/huggingface/optimum-intel) to speed up your SetFit models on Intel CPUs.\
Optimum Intel is the interface between the 🤗 Transformers and Diffusers libraries and the different tools and libraries provided by Intel to accelerate end-to-end pipelines on Intel architectures.

In our experiments, we will be using a fine-tuned SetFit model based on `bge-small-en-v1.5` sentence transformer model and observe our performance increase.

### Prerequisites

* Linux OS
* Python 3.9+
* An Intel Xeon Processor which supports [bfloat16](https://www.intel.com/content/www/us/en/developer/articles/technical/pytorch-on-xeon-processors-with-bfloat16.html)

Since Google Colab doesn't meet the hardware requirement, it's incompatible with this notebook and will not see speedups from our optimizations.

Please run the following to ensure you meet the requirements.

In [ ]:
import sys
import os
from time import sleep

os_info = !uname -o
os_type = os_info[0]
is_os_ok = "linux" in os_type.lower()
is_python_ver_ok = sys.version_info.major == 3 and sys.version_info.minor >=9
cpu_info = !lscpu
is_cpu_ok = "intel" in cpu_info.grep("vendor")[0].lower() and "bf16" in cpu_info.grep("flags")[0]
if not is_os_ok:
    print(f"This notebook requires a Linux type operating system, but you have {os_type}")
if not is_python_ver_ok:
    print(f"This notebook requires Python 3.9+ but your version is {sys.version.split()[0]}.")
if not is_cpu_ok:
    print(f"Your CPU does not support bfloat16.")
print(f"Notebook requirements {'' if is_os_ok and is_python_ver_ok and is_cpu_ok else 'not'} met.")

### Before Running this Notebook
1. Please create a `conda` environment and install `jupyterlab`:
```bash
conda create -n setfit python=3.10 -y && conda activate setfit
python -m pip install -U jupyterlab
```

2. Install **TCMalloc**: Google's fast, multi-threaded `malloc()` implementation:
```bash
conda install gperftools -c conda-forge -y
echo export LD_PRELOAD=${CONDA_PREFIX}/lib/libtcmalloc.so:$LD_PRELOAD >> ~/.bashrc
# **** Restart the shell or `source ~/.bashrc` ****
```

3. Launch the notebook using `numactl` (explained below), for example: 
```bash
OMP_NUM_THREADS=56 numactl -C 0-55 -m 0 jupyter lab
```

### Reproducibility
This notebook was tested on `Ubuntu 22.04.3 LTS` and `Python 3.10.13`.

To reproduce the maximum speeds reported in this notebook, please launch it using `numactl`.\
`numactl` enables gaining better control on which resources you're running. It's best to run on a single socket (`-m 0`):\
`OMP_NUM_THREADS=<NUM_THREADS> numactl -C <CORES_RANGE> -m 0 jupyter lab`

### Setup
Install the required packages to evaluate and visualize results for SetFit models:

In [ ]:
!python -m pip install -U setfit evaluate matplotlib -qqq

### Install Optimum Intel

Install Optimum Intel along with INC and IPEX backends which we'll use in our optimization later on.

In [ ]:
!python -m pip install --upgrade-strategy eager optimum[neural-compressor] -qqq
!python -m pip install intel-extension-for-pytorch -qqq

### Set Up Benchmark

First, define the infrastructure for conducting latency, throughput and accuracy benchmarks.

In [ ]:
from pathlib import Path
from time import perf_counter
import warnings
import subprocess
import sys
from collections import defaultdict

import evaluate
import numpy as np
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import trange
from sentence_transformers import SentenceTransformer
from setfit.exporters.utils import mean_pooling

warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def get_dir_size(path):
    ps = subprocess.Popen(('du','-hk', path), stdout=subprocess.PIPE)
    output = subprocess.check_output(("awk", "{print $1/1024,$2}"), stdin=ps.stdout)
    ps.wait()
    return float(output.split()[0].decode('utf-8'))

class PerformanceBenchmark:
    def __init__(self, model, dataset, optim_type, metric="accuracy", max_sequence_length=256,
                 model_path=None, enable_autocast=False, batch_sizes=None):
        self.model = model
        self.dataset = dataset
        self.optim_type = optim_type
        self.metric = evaluate.load(metric)
        self.model_path = model_path
        self.enable_autocast = enable_autocast
        self.tokenizer = self.model.model_body.tokenizer
        self.vocab_size = self.tokenizer.vocab_size
        self.max_length = max_sequence_length
        self.device = self.model.model_body.device
        self.batch_sizes = [2 ** i for i in range(12)] if batch_sizes is None else batch_sizes

    def to_device(self, batch):
        for key in batch:
            batch[key] = batch[key].to(self.device)
        return batch
        
    def timed_predict(self, inputs=None, encoded_inputs=None):
        if encoded_inputs is None:
            encoded_inputs = self.tokenizer(
            inputs, padding="max_length", max_length=self.max_length, truncation=True, return_tensors="pt"
        )
        start = perf_counter()
        encoded_inputs = self.to_device(encoded_inputs)
        with torch.no_grad(), torch.cpu.amp.autocast(dtype=torch.bfloat16, enabled=self.enable_autocast):
            if isinstance(self.model.model_body, SentenceTransformer):
                embeddings = self.model.model_body(encoded_inputs)["sentence_embedding"]
            else:
                outputs = self.model.model_body(**encoded_inputs)
                embeddings = mean_pooling(outputs["last_hidden_state"], encoded_inputs["attention_mask"])
            preds = self.model.model_head.predict(embeddings)
            latency = perf_counter() - start
        return preds, latency

    def compute_accuracy(self):
        preds, _ = self.timed_predict(self.dataset["text"])
        accuracy = self.metric.compute(predictions=preds, references=self.dataset["label"])
        print(f"Accuracy on test set - {accuracy['accuracy']:.3f}")
        return accuracy
        
    def compute_size(self):
        if self.model_path is None:
            state_dict = self.model.model_body.state_dict()
            tmp_path = Path("model.pt")
            torch.save(state_dict, tmp_path)
            size_mb = get_dir_size(tmp_path)
            # Delete temporary file
            tmp_path.unlink()
        else:
            size_mb = get_dir_size(self.model_path)
        print(f"Model size (MB) - {size_mb:.2f}")
        return {"size_mb": size_mb}

    def generate_random_sequences(self, batch_size):
        input_ids = torch.randint(0, self.vocab_size - 1, (batch_size, self.max_length))
        token_type_ids = torch.zeros((batch_size, self.max_length), dtype=torch.int64)
        attention_mask = torch.ones((batch_size, self.max_length), dtype=torch.int64)
        return {
            "input_ids": input_ids,
            "token_type_ids": token_type_ids,
            "attention_mask": attention_mask,
        }

    def compute_latency_and_throughput(self, num_iter=15, num_warmup_iter=5):
        res = defaultdict(list)
        print("=" * 100)
        for batch_size in self.batch_sizes:
            try:
                for _ in trange(num_warmup_iter, desc=f"Warmup ({batch_size=})", file=sys.stdout):
                    encoded_inputs = self.generate_random_sequences(batch_size=batch_size)
                    _ = self.timed_predict(encoded_inputs=encoded_inputs)

                batch_latencies = []
                for _ in trange(num_iter, desc=f"Timed Run ({batch_size=})", file=sys.stdout):
                    encoded_inputs = self.generate_random_sequences(batch_size=batch_size)
                    _, latency = self.timed_predict(encoded_inputs=encoded_inputs)
                    batch_latencies.append(latency)
        
                # Compute run statistics
                batch_latencies = np.array(batch_latencies)
                if batch_size == 1:
                    latency = 1000 * (batch_latencies / batch_size)
                    res.update({"time_avg_ms": np.mean(latency), "time_std_ms": np.std(latency)})
                    print(f"Avg. latency (ms), {batch_size=} - {res['time_avg_ms']:.2f} +/- {res['time_std_ms']:.2f}")

                throughputs = batch_size / batch_latencies
                res["throughputs_avg"].append(np.mean(throughputs))
                res["throughputs_std"].append(np.std(throughputs))
                print(f"Avg. throughput (samples/sec), {batch_size=} - {res['throughputs_avg'][-1]:.1f} +/- {res['throughputs_std'][-1]:.1f}")
                res["batch_sizes"].append(batch_size)
            except Exception as e:
                print(f"Skipping throughput computation at {batch_size=} since it failed with the following error: \n\n{e}\n")
                print(e.stack())
        return res
        
    def run_benchmark(self):
        all_metrics = {}
        for run_metric in  self.compute_latency_and_throughput, self.compute_size, self.compute_accuracy:
            all_metrics |= run_metric()
        return {self.optim_type: all_metrics}

Let's add a function to plot our results.

In [ ]:
def plot_metrics(perf_metrics):
    df = pd.DataFrame.from_dict(perf_metrics, orient="index")
    plt.figure(figsize=(6, 4), dpi=120)
    for idx in df.index:
        df_opt = df.loc[idx]
        plt.errorbar(
            df_opt["time_avg_ms"],
            df_opt["accuracy"] * 100,
            xerr=df_opt["time_std_ms"],
            fmt="o",
            alpha=0.5,
            ms=df_opt["size_mb"] / 15,
            label=idx,
            capsize=5,
            capthick=1,
        )
    legend = plt.legend(loc="lower right")
    plt.ylim(63, 95)
    # Use the slowest model to define the x-axis range
    xlim = max([metrics["time_avg_ms"] for metrics in perf_metrics.values()]) * 1.3
    plt.xlim(0, xlim)
    plt.ylabel("Accuracy (%)")
    plt.xlabel("Average latency with batch_size=1 (ms)")
    plt.show()

Load the dataset for running evaluations:

## 1. Benchmark SetFit using PyTorch

In [ ]:
from datasets import Dataset, load_dataset
test_dataset = load_dataset("SetFit/sst2")["validation"]

We'll now run the benchmark with the standard PyTorch backend:

In [ ]:
from setfit import SetFitModel

model_path = "dkorat/bge-small-en-v1.5_setfit-sst2-english"
small_model = SetFitModel.from_pretrained(model_path)
pb = PerformanceBenchmark(small_model, test_dataset, "bge-small (PyTorch)")
perf_metrics = pb.run_benchmark()

## 2. Optimize with Optimum Intel

In order to optimize our SetFit model, we will apply quantization to the model body, using Intel [Neural Compressor](https://www.intel.com/content/www/us/en/developer/tools/oneapi/neural-compressor.html) (INC), part of Optimum Intel.

**Quantization** is a very popular deep learning model optimization technique for improving inference speeds. It minimizes the number of bits required to represent the weights and/or activations in a neural network. This is done by converting a set of real-valued numbers into their lower-bit data representations, such as INT8. Moreover, quantization can enable faster computations in lower precision.

Specifically, we'll apply post-training static quantization (PTQ). PTQ can reduce the memory footprint and latency for inference, while still preserving the accuracy of the model, with only a small unlabeled calibration set and without any training.

#### Prepare a Calibration Dataset

The calibration dataset should be able to represent the data distribution of unseen data. In general, preparing 100 samples is enough for calibration. We'll use the Qasper dataset in our case.

In [ ]:
import random

def load_qasper_calibration_set(sample_size) -> Dataset:
    train_set = load_dataset("allenai/qasper")["train"]
    random.seed(666)
    random_samples = random.sample(range(len(train_set)), sample_size)
    random_queries = [random.sample(train_set[x]["qas"]["question"], 1)[0] for x in random_samples]
    random_abstracts = [train_set[x]["abstract"] for x in random_samples]
    samples = random.sample(random_queries + random_abstracts, sample_size)
    random.shuffle(samples)
    def gen():
        for s in samples:
            yield {"text": s}
    return Dataset.from_generator(gen)

#### Run Quantization
Define the desired quantization process using `optimum.intel`:

In [ ]:
import optimum.intel
from neural_compressor.config import PostTrainingQuantConfig

def quantize(setfit_model, output_path, calibration_set):
    def preprocess_function(examples):
        return tokenizer(examples["text"], padding="max_length", max_length=512, truncation=True)
        
    model = setfit_model.model_body[0].auto_model
    tokenizer = setfit_model.model_body.tokenizer
    vectorized_ds = calibration_set.map(preprocess_function, num_proc=10)
    vectorized_ds = vectorized_ds.remove_columns(["text"])
    quantizer = optimum.intel.INCQuantizer.from_pretrained(model)
    quantization_config = PostTrainingQuantConfig(approach="static", backend="ipex", domain="nlp")
    quantizer.quantize(
        quantization_config=quantization_config,
        calibration_dataset=vectorized_ds,
        save_directory=output_path,
        batch_size=1,
    )
    tokenizer.save_pretrained(output_path)

Quantize our SetFit model on 100 samples:

In [ ]:
model_path = "dkorat/bge-small-en-v1.5_setfit-sst2-english"
model = SetFitModel.from_pretrained(model_path)
calibration_set = load_qasper_calibration_set(sample_size=100)
optimum_model_path = "/tmp/bge-small-en-v1.5_setfit-sst2-english_opt"
quantize(model, output_path=optimum_model_path, calibration_set=calibration_set)

#### Benchmark Optimum Model

This next class defines a wrapper around our SetFit model which plugs in our quantized model at inference (instead of the original model body).

In [ ]:
import optimum.intel

class OptimumSetFitModel:
    def __init__(self, setfit_model, model_body):
        model_body.tokenizer = setfit_model.model_body.tokenizer
        self.model_body = model_body
        self.model_head = setfit_model.model_head

Time to run the performance benchmark on our optimized SetFit model!

In [ ]:
optimum_model = optimum.intel.IPEXModel.from_pretrained(optimum_model_path)
optimum_setfit_model = OptimumSetFitModel(model, model_body=optimum_model)

pb = PerformanceBenchmark(
    optimum_setfit_model,
    test_dataset,
    f"bge-small (optimum intel)",
    model_path=optimum_model_path,
    enable_autocast=True,
)
perf_metrics.update(pb.run_benchmark())

#### Plot Latencies and Compute Latency Speedup

In [ ]:
plot_metrics(perf_metrics)
speedup = perf_metrics['bge-small (PyTorch)']['time_avg_ms'] / perf_metrics['bge-small (optimum intel)']['time_avg_ms']
print(f"Latency speedup for 'bge-small (optimum-intel)': {speedup:.2f}x")

#### After optimizing, inference is 3.7x faster than before, with no drop in accuracy!

#### Throughput Speedup

Let's look now at the throughput, which is the number of samples the model can predict per second.\
We'll plot this value for our optimized and baseline models, as a function of the batch size.

In [ ]:
def plot_throughputs(perf_metrics):
    plt.figure(figsize=(10, 6), dpi=120)
    for (label, metric) in perf_metrics.items():
        xs = np.arange(len(metric["batch_sizes"]))
        y_mean, y_std = np.array(metric["throughputs_avg"]), np.array(metric["throughputs_std"])
        plt.plot(xs, y_mean, label=label, alpha=0.8)
        plt.fill_between(xs, y_mean - y_std, y_mean + y_std, alpha=0.2)
        for x, y in zip(xs, y_mean):
            plt.text(x, y, f'{y:.1f}', ha='right', va='bottom', fontsize=7, color="black")
        
    plt.xticks(xs, pb.batch_sizes[:len(xs)])
    plt.ylabel('Samples/Second')
    plt.suptitle('Throughput vs Batch Size')
    plt.title('\nBAAI/bge-small-en-v1.5 SetFit model trained on SetFit/sst2', fontsize=8)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

plot_throughputs(perf_metrics)

#### Similarly to the latency speedup, we can see that our optimization has resulted in up to 3.5x throughput increase as well!